<a href="https://colab.research.google.com/github/lialic/computer_vision/blob/master/Midterm/Sift_in_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scale-Invariant Feature Transform

With the same size of window, it can not detect Keypoint, especially with bigger edges

Bất biến với kích cỡ ảnh, dựa vào gradient của pixel, một vùng 8 chiều. Khi tăng cường độ sáng thì gradient ảnh hưởng. 

Được sử dụng để thay cho LoG khi mà nó hoạt độg như một máy dò phát hiện các đốm màu khách kích cỡ khi thay đổi $\sigma$.

Ở đây $\sigma$ được dùng như một tham số tỷ lệ, gaussian có sigma thấp cho giá trị cao với góc nhỏ, vì thế sigma cao sẽ phù hợp cho trường hợp này hơn.

Tuy nhiên LoG hơi tốn kém, vì thế sử dụng SIFT ở trường hợp này thì sự khác nhau giữa Gaussian và LoG không quá chênh lệch.

Cái khác cúa Gaussian là làm mờ hình ảnh với hai sigma khác nhau.

# Keypoint Localization 
Sau khi tìm được các Keypoint thì chúng phải được tinh chỉnh để có kết quả chính xác hơn.

Thứ nhất, sử dụng chuỗi Taylor mở rộng không gian tỷ lệ -> Keypoint chính xác hơn

Thứ hai, loại bỏ các cạnh (DoG có phản hồi cao với các cạnh) = cách sử dụng ma trận Hessian 2x2 (H) để dự đoán độ cong (giá trị eigen lớn hơn ở các cạnh vì thế nếu eigen lớn hơn một ngưỡng thì điểm đó sẽ bị loại bỏ)

# Mount drive and import libs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/Data/mnist_png.tar.gz", "/content/mnist")

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

## Unzip the .tar.gz file and load it into to the environment

## Adding file_name to image_path

In [ ]:
path = './mnist/mnist_png/training'

image_path = []

# Adding all 10 classes picture name into 
for i in range(10):
    dir = os.path.join(path, str(i))
    for file in os.listdir(dir):
        image_path.append(os.path.join(dir, file))

## Auxiliary function


In [ ]:
def CalcFeatures(img, th):

        '''
        Return descriptor which is used to append to the feature list
        Args:
        img - name of the image file
        th - threshhold 
        '''

        sift = cv2.xfeatures2d.SIFT_create(th)
        kp, des = sift.detectAndCompute(img, None)
        return des
def bag_of_features(features, centres, k = 500):
        '''
        The bag_of_features function assigns the features which are similar
        to a specific cluster centre thus forming a Bag of Words approach.  
        '''

        vec = np.zeros((1,k))
        for i in range(features.shape[0]):
            feat = features[i]
            diff = np.tile(feat, (k,1)) - centres
            dist = pow(((pow(diff, 2)).sum(axis = 1)), 0.5)
            idx_list = dist.argsort()
            idx = idx_list[0]
            vec[0][idx] +=1
        
        return vec

In [ ]:
def main(thresh):
    t0 = time.time()

    '''
    All the files appended to the image_path list are passed through the
    CalcFeatures functions which returns the descriptors which are 
    appended to the features list and then stacked vertically in the form
    of a numpy array.
    '''

    features = []
    for file in image_path:
        # Read the file as gray-scale image
        img = cv2.imread(file, 0)

        img_des = CalcFeatures(img, thresh)
        if img_des is not None:
            features.append(img_des)

    # Stack all the previous collected features into vertical np array  
    features = np.vstack(features)

    '''
    K-Means clustering is then performed on the feature array obtained 
    from the previous step. The centres obtained after clustering are 
    further used for bagging of features.
    '''

    k = 150
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    compactness, labels, centres = cv2.kmeans(features, k, None, criteria, 10 , flags)

    
    labels = []
    vec = []
    for file in image_path:
        img = cv2.imread(file, 0)
        img_des = CalcFeatures(img, thresh)

        if img_des is not None:
            img_vec = bag_of_features(img_des, centres, k)
            vec.append(img_vec)
            labels.append(int(file[27]))

    vec = np.vstack(vec)

    # Split data for trainning and testing SVM classifier
    x_train, x_test, y_train, y_test = train_test_split(vec, labels, test_size=0.2, random_state=0)

    clf = SVC()
    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    accuracy = accuracy_score(y_test, preds)
    conf_mat = confusion_matrix(y_test, preds)

    t1 = time.time()

    return accuracy * 100, conf_mat, (t1-t0)



In [ ]:
accuracy = []
timer = []
for i in range(5, 26, 5):
    print('\nCalculating for a threshold of {}'.format(i))
    data = main(i)
    accuracy.append(data[0])
    conf_mat = data[1]
    timer.append(data[2])
    print('\nAccuracy = {}\nTime taken = {} sec\nConfusion matrix :\n{}'.format(data[0],data[2],data[1]))


Calculating for a threshold of 5

Accuracy = 67.22388059701493
Time taken = 828.3164970874786 sec
Confusion matrix :
[[ 996    6   22   11    3   27   36   12   24   18]
 [   0 1116   11    4   15    5    3   13    2    9]
 [  65   12  584   95   74   43   31  166   71   29]
 [  36    0  151  725   49  122   16   53   29   23]
 [  10   10   88   32  794   41   25   59   43   82]
 [  93    7   32   71   39  681   61   51   22   37]
 [  98   18   47   15   14   44  608   66   49  214]
 [  22   67  112   37   38   14   43  854   11   14]
 [  61   10   51   29   44   33   50   11  812   58]
 [  44    5   35   29   52   67  160   19   73  712]]

Calculating for a threshold of 10

Accuracy = 76.07675906183368
Time taken = 911.0088083744049 sec
Confusion matrix :
[[1011    5   17    6    0   13   51   23   12   17]
 [   0 1122    9    1   15    5    6   15    3    2]
 [  39    7  798   45   24   29   26  149   41   12]
 [  14    0   81  887   20  106   21   31   34   10]
 [   7    4   35   1